# Выбор локации для скважины

## Описание задачи и данных

**Задача:**

Компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину. Имеются пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Необходимо построить модель для определения региона, где добыча принесёт наибольшую прибыль.

**Описание данных:**
* `id` — уникальный идентификатор скважины

* `f0, f1, f2` — три признака точек (неважно, что они означают, но сами признаки значимы)

* `product` — объём запасов в скважине (тыс. баррелей)

**Условия:**
* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

* Бюджет на разработку скважин в регионе — 10 млрд рублей.

* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Импорт библиотек, загрузка данных и изучение общей информации о них

In [23]:
import pandas as pd
import numpy as np
from numpy.random import RandomState

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression

In [49]:
reg_1 = pd.read_csv('geo_data_0.csv')
reg_2 = pd.read_csv('geo_data_1.csv')
reg_3 = pd.read_csv('geo_data_2.csv')

In [50]:
reg_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [51]:
reg_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [52]:
reg_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [53]:
reg_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [54]:
reg_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [55]:
reg_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


#### Работа с дубликатами

In [56]:
duplicated_ids = reg_1.loc[reg_1['id'].duplicated(), 'id']
for i in duplicated_ids:
    print(reg_1[reg_1['id'] == i])
    print()

         id        f0        f1         f2     product
931   HZww2  0.755284  0.368511   1.863211   30.681774
7530  HZww2  1.061194 -0.373969  10.430210  158.828695

          id        f0        f1        f2    product
1364   bxg6G  0.411645  0.856830 -3.653440  73.604260
41724  bxg6G -0.823752  0.546319  3.630479  93.007798

          id        f0        f1        f2    product
3389   A5aEY -0.039949  0.156872  0.209861  89.249364
51970  A5aEY -0.180335  0.935548 -2.094773  33.020205

          id        f0        f1        f2    product
1949   QcMuo  0.506563 -0.323775 -2.215583  75.496502
63593  QcMuo  0.635635 -0.473422  0.862670  64.578675

          id        f0        f1        f2     product
64022  74z30  0.741456  0.459229  5.153109  140.771492
66136  74z30  1.084962 -0.312358  6.990771  127.643327

          id        f0        f1        f2     product
42529  AGS9W  1.454747 -0.479651  0.683380  126.370504
69163  AGS9W -0.933795  0.116194 -3.655896   19.230453

          id 

В описании данных сказано, что столбец `'id'` содержит уникальный идентификатор скважины. Странно, что в файле `geo_data_0.csv`
некоторые скважины встречаются дважды... Удалим их

In [57]:
for i in duplicated_ids:
    reg_1 = reg_1.loc[reg_1['id'] != i].reset_index(drop=True)

Аналогичным образом поступим и с другими двумя датафреймами

In [58]:
duplicated_ids = reg_2.loc[reg_2['id'].duplicated(), 'id']
for i in duplicated_ids:
    print(reg_2[reg_2['id'] == i])
    print()

          id         f0        f1        f2    product
1305   LHZR0  11.170835 -1.945066  3.002872  80.859783
41906  LHZR0  -8.989672 -4.286607  2.009139  57.085625

          id        f0        f1        f2     product
2721   bfPNe -9.494442 -5.463692  4.006042  110.992147
82178  bfPNe -6.202799 -4.820045  2.995107   84.038886

          id         f0        f1        f2     product
47591  wt4Uk  -9.091098 -8.109279 -0.002314    3.179103
82873  wt4Uk  10.259972 -9.376355  4.994297  134.766305

          id         f0         f1        f2     product
5849   5ltQ6  -3.435401 -12.296043  1.999796   57.085625
84461  5ltQ6  18.213839   2.191999  3.993869  107.813044



In [59]:
for i in duplicated_ids:
    reg_2 = reg_2.loc[reg_2['id'] != i].reset_index(drop=True)

In [60]:
duplicated_ids = reg_3.loc[reg_3['id'].duplicated(), 'id']
for i in duplicated_ids:
    print(reg_3[reg_3['id'] == i])
    print()

          id        f0        f1        f2     product
28039  xCHr8  1.633027  0.368135 -2.378367    6.120525
43233  xCHr8 -0.847066  2.101796  5.597130  184.388641

          id        f0        f1        f2     product
11449  VF7Jo  2.122656 -0.858275  5.746001  181.716817
49564  VF7Jo -0.883115  0.560537  0.723601  136.233420

          id        f0        f1        f2     product
45404  KUPhW  0.231846 -1.698941  4.990775   11.716299
55967  KUPhW  1.211150  3.176408  5.543540  132.831802

          id        f0        f1        f2     product
44378  Vcm5J -1.229484 -2.439204  1.222909  137.968290
95090  Vcm5J  2.587702  1.986875  2.482245   92.327572



In [61]:
for i in duplicated_ids:
    reg_3 = reg_3.loc[reg_3['id'] != i].reset_index(drop=True)

## Подготовка данных и обучение моделей поочереди для всех регионов

В наших данных нет пропусков, что уже хорошо. Категориальных столбцов у нас тоже нет, однако есть числовые: `f0, f1, f2`. Попробуем их масштабировать и для каждого региона посмотрим, какая модель лучше справляется: которая учится на масштабированных данных или которая учится на "сырых" данных. И последнее, касающееся предобработки: столбец, содержащий ID скважины не является информативным, поэтому использовать его для обучения моделей смысла нет.

<div class="alert alert-block alert-info">
<b>Совет:</b>  А изучить распределения данных? А корреляции? Изучение корреляций в данных очень важно для линейных моделей, потому что оно помогает оценить силу и направление связи между различными переменными в данных. Это может помочь избежать включения в модель переменных с высокой корреляцией между обучающими данными, что может привести к переобучению и потере точности предсказаний. Также изучение корреляций может помочь выбрать наиболее значимые переменные для включения в модель. 
</div>

In [62]:
reg_1 = reg_1.drop('id', axis=1)
reg_2 = reg_2.drop('id', axis=1)
reg_3 = reg_3.drop('id', axis=1)

### Функция для обучения модели

In [63]:
# Данная функция предполагалась более-менее универсальной, то есть она включает в себя не только обучение модели,
# но и возможности масштабирования разными методами (параметр scale=True; scaler="Standard" или "MinMax" - использование
# StandardScaler или MinMaxScaler, соответственно)
#
# Возвращает: правильные ответы на валидационной выборке, предсказания модели, RMSE, r2_score и саму модель
def LinearRegressionFitting(features, target, scale=True, scaler='Standard'):
    # Разбиваем на обучающую и валидационную выборки
    X_train, X_valid, y_train, y_valid = train_test_split(features, target, test_size=0.25, random_state=42)
    
    # При необходимости масштабируем
    if scale:
        if scaler == 'Standard':
            scaler = StandardScaler(with_mean=True)
        else:
            scaler = MinMaxScaler()
        
        X_train = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=['f0', 'f1', 'f2']
        )
        X_valid = pd.DataFrame(
            scaler.transform(X_valid),
            columns=['f0', 'f1', 'f2']
        )
    
    # Обучаем модель, выполненяем предсказания и подсчет метрик
    model = LinearRegression()
    model.fit(X_train, y_train)
    pred = model.predict(X_valid)
    
    rmse = mean_squared_error(y_valid, pred)**0.5
    r2 = r2_score(y_valid, pred)
    
    # Предсказание константной модели (всегда предсказывает медианное значение из обучающего набора)
    const_model_pred = [y_train.median()] * y_valid.shape[0]
    
    return y_valid, pred, const_model_pred, rmse, r2, model

### Регион 1 (файл geo_data_0.csv)

In [64]:
X, y = reg_1.drop('product', axis=1), reg_1['product']

In [65]:
# Без масштабирования
y_valid, pred, const_model_pred, rmse, r2, model = LinearRegressionFitting(X, y, scale=False)
print('Средний запас предсказанного сырья:', np.mean(pred))
print('RMSE:', rmse)
print('r2_score:', r2)
print()
print('RMSE константной модели:', mean_squared_error(y_valid, const_model_pred)**0.5)

Средний запас предсказанного сырья: 92.2865385223536
RMSE: 37.76114714593895
r2_score: 0.2720543226893649

RMSE константной модели: 44.25865263278167


In [66]:
# Пробуем StandardScaler
y_valid, pred, const_model_pred, rmse, r2, model = LinearRegressionFitting(X, y)
print('Средний запас предсказанного сырья:', np.mean(pred))
print('RMSE:', rmse)
print('r2_score:', r2)
print()
print('RMSE константной модели:', mean_squared_error(y_valid, const_model_pred)**0.5)

Средний запас предсказанного сырья: 92.2865385223536
RMSE: 37.76114714593895
r2_score: 0.2720543226893649

RMSE константной модели: 44.25865263278167


In [67]:
# Пробуем MinMaxScaler
y_valid, pred, const_model_pred, rmse, r2, model = LinearRegressionFitting(X, y, scale=True, scaler='MinMax')
print('Средний запас предсказанного сырья:', np.mean(pred))
print('RMSE:', rmse)
print('r2_score:', r2)
print()
print('RMSE константной модели:', mean_squared_error(y_valid, const_model_pred)**0.5)

Средний запас предсказанного сырья: 92.2865385223536
RMSE: 37.76114714593895
r2_score: 0.2720543226893649

RMSE константной модели: 44.25865263278167


### Регион 2 (файл geo_data_1.csv)

In [68]:
X, y = reg_2.drop('product', axis=1), reg_2['product']

In [69]:
# Без масштабирования
y_valid, pred, const_model_pred, rmse, r2, model = LinearRegressionFitting(X, y, scale=False)
print('Средний запас предсказанного сырья:', np.mean(pred))
print('RMSE:', rmse)
print('r2_score:', r2)
print()
print('RMSE константной модели:', mean_squared_error(y_valid, const_model_pred)**0.5)

Средний запас предсказанного сырья: 69.16894101447984
RMSE: 0.892863589877732
r2_score: 0.9996227439716872

RMSE константной модели: 47.52975727437665


In [70]:
# Пробуем StandardScaler
y_valid, pred, const_model_pred, rmse, r2, model = LinearRegressionFitting(X, y)
print('Средний запас предсказанного сырья:', np.mean(pred))
print('RMSE:', rmse)
print('r2_score:', r2)
print()
print('RMSE константной модели:', mean_squared_error(y_valid, const_model_pred)**0.5)

Средний запас предсказанного сырья: 69.16894101447984
RMSE: 0.892863589877732
r2_score: 0.9996227439716872

RMSE константной модели: 47.52975727437665


In [71]:
# Пробуем MinMaxScaler
y_valid, pred, const_model_pred, rmse, r2, model = LinearRegressionFitting(X, y, scale=True, scaler='MinMax')
print('Средний запас предсказанного сырья:', np.mean(pred))
print('RMSE:', rmse)
print('r2_score:', r2)
print()
print('RMSE константной модели:', mean_squared_error(y_valid, const_model_pred)**0.5)

Средний запас предсказанного сырья: 69.16894101447987
RMSE: 0.8928635898777322
r2_score: 0.9996227439716872

RMSE константной модели: 47.52975727437665


### Регион 3 (файл geo_data_2.csv)

In [72]:
X, y = reg_3.drop('product', axis=1), reg_3['product']

In [73]:
# Без масштабирования
y_valid, pred, const_model_pred, rmse, r2, model = LinearRegressionFitting(X, y, scale=False)
print('Средний запас предсказанного сырья:', np.mean(pred))
print('RMSE:', rmse)
print('r2_score:', r2)
print()
print('RMSE константной модели:', mean_squared_error(y_valid, const_model_pred)**0.5)

Средний запас предсказанного сырья: 94.82069136328788
RMSE: 40.13115560413051
r2_score: 0.19488984966240108

RMSE константной модели: 44.72575932729128


In [74]:
# Пробуем StandardScaler
y_valid, pred, const_model_pred, rmse, r2, model = LinearRegressionFitting(X, y)
print('Средний запас предсказанного сырья:', np.mean(pred))
print('RMSE:', rmse)
print('r2_score:', r2)
print()
print('RMSE константной модели:', mean_squared_error(y_valid, const_model_pred)**0.5)

Средний запас предсказанного сырья: 94.82069136328788
RMSE: 40.131155604130505
r2_score: 0.1948898496624013

RMSE константной модели: 44.72575932729128


In [75]:
# Пробуем MinMaxScaler
y_valid, pred, const_model_pred, rmse, r2, model = LinearRegressionFitting(X, y, scale=True, scaler='MinMax')
print('Средний запас предсказанного сырья:', np.mean(pred))
print('RMSE:', rmse)
print('r2_score:', r2)
print()
print('RMSE константной модели:', mean_squared_error(y_valid, const_model_pred)**0.5)

Средний запас предсказанного сырья: 94.82069136328789
RMSE: 40.13115560413051
r2_score: 0.19488984966240108

RMSE константной модели: 44.72575932729128


**Вывод:**

Видно, что масштабирование данных не повышает точность модели. 

Средний запас предсказанного сырья (тыс баррелей) по регионам:

* Регион 1: 92.29
* Регион 2: 69.17
* Регион 3: 94.82

Также хочется отметить, что единственный регион, который хорошо предсказывается нашей моделью - регион 2. На регионах 1 и 3 метрики RMSE и R2 оставляют желать лучшего. Однако все модели справляются со своей задачей лучше, чем константные, предсказывающие медиану обучающего набора.

## Подготовка к рассчету прибыли

Сохраним некоторые важные для вычисления прибыли значения в отдельных переменных

In [76]:
budget = 10**10  # бюджет на разработку 200 скважин (в рублях)
cost = 450_000  # стоимость единицы продукта (в рублях)

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины и сравним полученный объём сырья со средним запасом в каждом регионе

In [77]:
min_volume = budget / cost / 200  # минимальный достаточный объем для безубыточной разработки новой скважины (в тыс баррелей)
print(f'{min_volume : .2f}')

 111.11


In [78]:
print(f"Средний запас сырья в первом регионе: {reg_1['product'].mean() : .2f}")
print(f"Средний запас сырья во втором регионе: {reg_2['product'].mean() : .2f}")
print(f"Средний запас сырья в третьем регионе: {reg_3['product'].mean() : .2f}")

Средний запас сырья в первом регионе:  92.50
Средний запас сырья во втором регионе:  68.82
Средний запас сырья в третьем регионе:  95.00


**Вывод:**

Минимальный достаточный объем сырья для безубыточной разработки скважины составляет примерно 111 тыс баррелей. Это выше среднего запаса в скважине в каждом регионе.

#### Функция для рассчета прибыли

In [79]:
def profit_calc(data, predictions, budget=budget, cost=cost):
    data = data
    data['prediction'] = predictions
    data = data.sort_values(by='prediction', ascending=False)[:200]
    volume = data['product'].sum()
    profit = volume * cost - budget
    return profit

## Рассчет прибыли и рисков с помощью техники Bootstrap

In [80]:
# Функция для применения техники bootstrap для рассчета прибыли
def Bootstrap(reg):
    # Обучение модели для соответствующего региона
    X, y = reg.drop('product', axis=1), reg['product']
    y_valid, pred, const_model_pred, rmse, r2, model = LinearRegressionFitting(X, y)
    
    # Собственно bootstrap
    profits = []
    state = RandomState(42)
    for i in range(1000):
        # Делаем выборку из 500 точек
        subsample = reg.sample(n=500, replace=True, random_state=state)

        # Выполняем предсказания объема сырья в этих 500 точках и выбираем лучшие 200 по мнению модели
        features = subsample.drop('product', axis=1)
        pred = model.predict(features)

        # Считаем прибыль
        profit = profit_calc(subsample, pred)
        profits.append(profit)

    profits = pd.Series(profits)
    
    # Считаем среднюю прибыль, 95%-й доверительный интервал для прибыли и риски убытков
    mean_profit = profits.mean()
    lower, upper = round(profits.quantile(0.025), 2), round(profits.quantile(0.975), 2)
    risk = (profits < 0).astype('int').mean() * 100
    
    return mean_profit, (lower, upper), risk

In [81]:
regions = [reg_1, reg_2, reg_3]
for i in range(3):
    mean_profit, confidence_interval, risk = Bootstrap(regions[i])
    print(f'Регион {i + 1}')
    print(f'Средняя прибыль: {mean_profit:.2f}')
    print(f'95%-й доверительный интервал для прибыли: {confidence_interval}')
    print(f'Риски убытков: {risk:.1f}%')
    print('--------------------------------------------------')

Регион 1
Средняя прибыль: 335511635.70
95%-й доверительный интервал для прибыли: (-153419786.97, 865119610.28)
Риски убытков: 9.6%
--------------------------------------------------
Регион 2
Средняя прибыль: 454097691.21
95%-й доверительный интервал для прибыли: (48196445.84, 845190897.91)
Риски убытков: 1.2%
--------------------------------------------------
Регион 3
Средняя прибыль: 371762620.36
95%-й доверительный интервал для прибыли: (-128004534.02, 880666247.08)
Риски убытков: 8.2%
--------------------------------------------------


## Вывод

1) При подготовке моделей для каждого региона выяснилось, что масштабирование признаков не влияет на результат

2) Единственный регион, который хорошо описывается линейной регрессией - это регион 2 (файл `geo_data_1.csv`), однако все модели справляются со своей задачей лучше, чем константная модель, предсказывающая медианное значение

3) Средний запас предсказанного сырья в скважине(тыс баррелей) по регионам:

* Регион 1: 92.29
* Регион 2: 69.17
* Регион 3: 94.82

4) Для каждого региона посчитаны средняя прибыль (на 200 скважин), 95%-й доверительный интервал для прибыли и риски убытков. Результаты:

* Регион 1
  * Средняя прибыль: 335511635.7
  * 95%-й доверительный интервал для прибыли: (-153419786.97, 865119610.28)
  * Риски убытков: 9.6%

* Регион 2
  * Средняя прибыль: 454097691.21
  * 95%-й доверительный интервал для прибыли: (48196445.84, 845190897.91)
  * Риски убытков: 1.2%

* Регион 3
  * Средняя прибыль: 371762620.36
  * 95%-й доверительный интервал для прибыли: (-128004534.02, 880666247.08)
  * Риски убытков: 8.2%

5) По результатам исследования можно предложить для бурения новой скважины регион 2 (файл `geo_data_1.csv`). Основания: самая высокая средняя прибыль, 95%-й доверительный интервал не содержит отрицательных значений, риски убытков самые низкие среди исследованных регионов и составляют всего 1.2%, помимо всего прочего, это единственный регион, который хорошо описывается разрешенными моделями (линейной регрессией)